In [39]:
import os
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import numpy as np


In [2]:
iskaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")
comp_name = "us-patent-phrase-to-phrase-matching"
comp_path = Path(comp_name)

In [5]:
if not iskaggle:
    import kaggle, zipfile
    kaggle.api.competition_download_cli(comp_name)
    zipfile.ZipFile(f"{comp_name}.zip").extractall(comp_name)
else:
    comp_path = Path("../input/us-patent-phrase-to-phrase-matching")

us-patent-phrase-to-phrase-matching.zip: Skipping, found more recently modified local copy (use --force to force download)


In [35]:
df = pd.read_csv(comp_path/"train.csv")
df_test = pd.read_csv(comp_path/"test.csv")
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [9]:
df.describe(include=object)

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [36]:
df["input"] = "CONTEXT[" + df.context + "] + TARGET[" + df.target + "] + ANCHOR[" + df.anchor + "]"
df_test["input"] = "CONTEXT[" + df_test.context + "] + TARGET[" + df_test.target + "] + ANCHOR[" + df_test.anchor + "]"
df.input[0]

'CONTEXT[A47] + TARGET[abatement of pollution] + ANCHOR[abatement]'

In [37]:
ds = Dataset.from_pandas(df)
ds_test = Dataset.from_pandas(df_test)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

## deberta-v3-small

In [21]:
model_name = "microsoft/deberta-v3-small"

In [22]:
tokz = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
df.input[0], list(zip(['[CLS]'] + tokz.tokenize(df.input[0]) + ['[SEP]'], tokz(df.input[0])["input_ids"]))

('CONTEXT[A47] + TARGET[abatement of pollution] + ANCHOR[abatement]',
 [('[CLS]', 1),
  ('▁CON', 20967),
  ('TEXT', 104917),
  ('[', 2550),
  ('A', 558),
  ('47', 5753),
  (']', 592),
  ('▁+', 1566),
  ('▁TARGET', 90632),
  ('[', 2550),
  ('aba', 16191),
  ('t', 297),
  ('ement', 17019),
  ('▁of', 265),
  ('▁pollution', 6435),
  (']', 592),
  ('▁+', 1566),
  ('▁ANCHOR', 99793),
  ('[', 2550),
  ('aba', 16191),
  ('t', 297),
  ('ement', 17019),
  (']', 592),
  ('[SEP]', 2)])

In [26]:
def tok_func(ds): return tokz(ds["input"])
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [28]:
tok_ds[0]["input"], tok_ds[0]["input_ids"]

('CONTEXT[A47] + TARGET[abatement of pollution] + ANCHOR[abatement]',
 [1,
  20967,
  104917,
  2550,
  558,
  5753,
  592,
  1566,
  90632,
  2550,
  16191,
  297,
  17019,
  265,
  6435,
  592,
  1566,
  99793,
  2550,
  16191,
  297,
  17019,
  592,
  2])

In [ ]:
tok_ds = tok_ds.rename_columns({"score": "labels"})

In [34]:
ds_dict = tok_ds.train_test_split(0.175, seed=42)
ds_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30090
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6383
    })
})

In [41]:
def corr(x, y): return np.corrcoef(x, y)[0, 1]
def corr_d(eval_preds): return {"pearson": corr(*eval_preds)}

In [42]:
bs = 128
epochs = 4
lr = 8e-5

In [48]:
args = TrainingArguments(
    f"out_{model_name}", 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type="cosine",
    fp16=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to="none"
)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=ds_dict["train"], eval_dataset=ds_dict["test"],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [49]:
trainer.train();

/home/nblzv/mambaforge/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.034180,0.757738
2,No log,0.026765,0.801307
3,0.038700,0.023140,0.814804
4,0.038700,0.024084,0.815799


## gpt2-small

In [50]:
model_name = "gpt2"

In [70]:
tokz = AutoTokenizer.from_pretrained(model_name)
tokz.pad_token = tokz.eos_token

In [71]:
df.input[0], list(zip(tokz.tokenize(df.input[0]), tokz(df.input[0])["input_ids"]))

('CONTEXT[A47] + TARGET[abatement of pollution] + ANCHOR[abatement]',
 [('CON', 10943),
  ('TEXT', 32541),
  ('[', 58),
  ('A', 32),
  ('47', 2857),
  (']', 60),
  ('Ġ+', 1343),
  ('ĠT', 309),
  ('ARGET', 46095),
  ('[', 58),
  ('ab', 397),
  ('at', 265),
  ('ement', 972),
  ('Ġof', 286),
  ('Ġpollution', 12231),
  (']', 60),
  ('Ġ+', 1343),
  ('ĠAN', 3537),
  ('CH', 3398),
  ('OR', 1581),
  ('[', 58),
  ('ab', 397),
  ('at', 265),
  ('ement', 972),
  (']', 60)])

In [72]:
def tok_func(ds): return tokz(ds["input"])
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [73]:
tok_ds[0]["input"], tok_ds[0]["input_ids"]

('CONTEXT[A47] + TARGET[abatement of pollution] + ANCHOR[abatement]',
 [10943,
  32541,
  58,
  32,
  2857,
  60,
  1343,
  309,
  46095,
  58,
  397,
  265,
  972,
  286,
  12231,
  60,
  1343,
  3537,
  3398,
  1581,
  58,
  397,
  265,
  972,
  60])

In [ ]:
tok_ds = tok_ds.rename_columns({"score": "labels"})

In [132]:
ds_dict = tok_ds.train_test_split(0.175, seed=42)
ds_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
        num_rows: 30090
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
        num_rows: 6383
    })
})

In [133]:
def corr(x, y): return np.corrcoef(x.squeeze(), y)[0, 1]
def corr_d(eval_preds): return {"pearson": corr(*eval_preds)}

In [134]:
bs = 128
epochs = 4
lr = 8e-5

In [135]:
args = TrainingArguments(
    f"out_{model_name}", 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type="cosine",
    fp16=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to="none"
)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
model.config.pad_token_id = model.config.eos_token_id
trainer = Trainer(model, args, train_dataset=ds_dict["train"], eval_dataset=ds_dict["test"],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [136]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.038990,0.650291
2,No log,0.036460,0.718685
3,0.077500,0.029349,0.746914
4,0.077500,0.029363,0.753194
